In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import random
import os

In [ ]:

### Directory paths
DATA_PATH = './data/original'
AUGMENTED_PATH = './data/rnn_augmented'
RNN_BALANCED_PATH = './data/rnn_balanced'
MODELS_PATH = './models/rnn'

TRAIN_DIRECTORY='train'
TEST_DIRECTORY='test'
VALIDATION_DIRECTORY ='val'

### Sample Values
TRAIN_DATASET_SAMPLE_COUNT=1000
TEST_DATASET_SAMPLE_COUNT=200
VALIDATION_DATASET_SAMPLE_COUNT=200

### Directories containing the images
SUB_DIRECTORIES = [TRAIN_DIRECTORY, TEST_DIRECTORY, VALIDATION_DIRECTORY]
DATA_DIRECTORIES = ['Actinic keratoses', 'Basal cell carcinoma', 'Benign keratosis-like lesions', 'Chickenpox', 'Cowpox', 'Dermatofibroma', 'Healthy', 'HFMD', 'Measles', 'Melanocytic nevi', 'Melanoma', 'Monkeypox', 'Squamous cell carcinoma', 'Vascular lesions']

### Data generation properties
ROTATION_RANGE = 40
WIDTH_SHIFT_RANGE = 0.2
HEIGHT_SHIFT_RANGE = 0.2
SHEAR_RANGE = 0.2
ZOOM_RANGE = 0.2
HORIZONTAL_FLIP = True
FILL_MODE = 'nearest'

IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = len(DATA_DIRECTORIES)
BATCH_SIZE = 32
EPOCHS = 20

In [ ]:
### This method is used to get the number of images in each of the directories in our dataset

def count_images(dir_name):
    for dataset_type in SUB_DIRECTORIES:
        dir_type = os.path.join(dir_name, dataset_type)
        print(f"{dataset_type}")
        print("----------------------------")
        for category in os.listdir(dir_type):
            category_path = os.path.join(dir_type, category)
            if not os.path.isdir(category_path):
                continue
            images = [img for img in os.listdir(category_path) if img.endswith(('jpg', 'jpeg'))]
            print(f"Number of images in {category_path.split('/')[-1]}: {len(images)}")
        print("")

In [ ]:
### Execute the method

count_images(DATA_PATH)

In [ ]:
## 🗜️ Image Augmentation for Categories with Low Image Count

### Define directories that lack the images count
TRAIN_IMAGE_AUGEMENTATION_DIRS = ['Cowpox', 'Actinic keratoses', 'Measles', 'Chickenpox', 'Squamous cell carcinoma', 'Dermatofibroma', 'Vascular lesions']
TEST_IMAGE_AUGEMENTATION_DIRS = ['Cowpox', 'Healthy', 'Actinic keratoses', 'Measles', 'Chickenpox', 'Squamous cell carcinoma', 'Dermatofibroma', 'Vascular lesions']
VAL_IMAGE_AUGEMENTATION_DIRS = ['Cowpox', 'Healthy', 'Actinic keratoses', 'Measles', 'Chickenpox', 'Squamous cell carcinoma', 'Dermatofibroma', 'Vascular lesions']

### Create Image Generator
datagen = ImageDataGenerator(
    rotation_range=ROTATION_RANGE,
    width_shift_range=WIDTH_SHIFT_RANGE,
    height_shift_range=HEIGHT_SHIFT_RANGE,
    shear_range=SHEAR_RANGE,
    zoom_range=ZOOM_RANGE,
    horizontal_flip=HORIZONTAL_FLIP,
    fill_mode=FILL_MODE)

### Function to process and augment data
def augment_and_copy_data(dir_name):
    for dataset_type in SUB_DIRECTORIES:
        dir_type = os.path.join(dir_name, dataset_type)
        ### Dependeing on the dataset_type pick the categories with less number of images
        categories = []
        if dataset_type == TRAIN_DIRECTORY:
            categories = TRAIN_IMAGE_AUGEMENTATION_DIRS
        elif dataset_type == TEST_DIRECTORY:
            categories = TEST_IMAGE_AUGEMENTATION_DIRS
        else:
            categories = VAL_IMAGE_AUGEMENTATION_DIRS
        for category in categories:
            category_path = os.path.join(dir_type, category)
            ### Get augmented dataset path (e.g. ./data/augmented/train)
            augmented_path = os.path.join(AUGMENTED_PATH, dataset_type)
            ### Create the dataset type directory if it does not exist
            os.makedirs(augmented_path, exist_ok=True)
            ### Get augmented category path (e.g. ./data/augmented/train/Cowpox)
            augmented_category_path = os.path.join(augmented_path, category)
            ### Create a directory if it does not exist
            os.makedirs(augmented_category_path, exist_ok=True)

            ### Get all the files inside a category directory
            files = os.listdir(category_path)

            print(f'Started augmenting images in: {category_path}')

            ### Iterate files
            for file in files:
                if file.startswith('.'): ### Skip hidden files
                    continue
                ### Load images
                img_path = os.path.join(category_path, file)
                img = load_img(str(img_path))
                x = img_to_array(img) ### This is a NumPy array with shape (3, 150, 150)
                x = x.reshape((1,) + x.shape) ### This is a NumPy array with shape (1, 3, 150, 150)
                ### The .flow() command generates batches of randomly transformed images and saves the results in the ./data/augmented/<sub_directory>/<data_directory> along with the original image
                ### print(f'Started augmenting image: {img_path}')
                i = 0
                for batch in datagen.flow(x, batch_size=1, save_to_dir=augmented_category_path, save_prefix=file.split(".")[0], save_format='jpg'):
                    i += 1
                    if i > 10:
                        break ### Stop looping the generator infinitely

In [ ]:
### Execute the method

augment_and_copy_data(DATA_PATH)

In [ ]:
## ⚖️ Create a Balanced Dataset
### This method is used to copy a certain number of images from every, train, test, and validation set.

def limit_data_from_source(source_dir, dest_dir, num_images=TRAIN_DATASET_SAMPLE_COUNT):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    for dataset_type in SUB_DIRECTORIES:
        source_type_dir = os.path.join(source_dir, dataset_type)
        dest_type_dir = os.path.join(dest_dir, dataset_type)
        os.makedirs(dest_type_dir, exist_ok=True)
        for category in os.listdir(source_type_dir):
            category_path = os.path.join(source_type_dir, category)
            if not os.path.isdir(category_path):
                continue
            dest_category_path = os.path.join(dest_type_dir, category)
            os.makedirs(dest_category_path, exist_ok=True)

            images = [img for img in os.listdir(category_path) if img.endswith(('jpg', 'jpeg'))]
            print(f"Number of images in {dataset_type}/{category_path.split('/')[-1]}: {len(images)}")
            ### Select the number of images provided in num_images param or the minimum number of images
            if dataset_type == TRAIN_DIRECTORY:
                num_images = TRAIN_DATASET_SAMPLE_COUNT
            elif dataset_type == TEST_DIRECTORY:
                num_images = TEST_DATASET_SAMPLE_COUNT
            else:
                num_images = VALIDATION_DATASET_SAMPLE_COUNT
            selected_images = random.sample(images, min(num_images, len(images)))

            for image in selected_images:
                source_image_path = os.path.join(category_path, image)
                dest_image_path = os.path.join(dest_category_path, image)
                ### print(f'Copying {source_image_path} to {dest_image_path}')
                shutil.copy(source_image_path, dest_image_path)          

In [ ]:
limit_data_from_source(DATA_PATH, RNN_BALANCED_PATH)

In [ ]:
count_images(CNN_BALANCED_PATH)

In [ ]:
## Since there are missing images, we need to find out exactly how many images we are missing by creating dictiionaries for each of the test, train, and val directories.
## Function for getting missing image count for train, test, and val

def get_missing_images_count(dir_name):
    ## Create dictionaries for train, test, and validation
    train_dict = dict()
    test_dict = dict()
    val_dict = dict()
    for dataset_type in SUB_DIRECTORIES:
        dir_type = os.path.join(dir_name, dataset_type)
        for category in os.listdir(dir_type):
            category_path = os.path.join(dir_type, category)
            if not os.path.isdir(category_path):
                continue
            images = [img for img in os.listdir(category_path) if img.endswith(('jpg', 'jpeg'))]
            required_count = 0
            if dataset_type == TRAIN_DIRECTORY:
                image_count = min(len(images), TRAIN_DATASET_SAMPLE_COUNT)
                required_count = TRAIN_DATASET_SAMPLE_COUNT - image_count
                train_dict[category_path.split("/")[-1]] = required_count
            elif dataset_type == TEST_DIRECTORY:
                image_count = min(len(images), TEST_DATASET_SAMPLE_COUNT)
                required_count = TEST_DATASET_SAMPLE_COUNT - image_count
                test_dict[category_path.split("/")[-1]] = required_count
            else:
                image_count = min(len(images), VALIDATION_DATASET_SAMPLE_COUNT)
                required_count = VALIDATION_DATASET_SAMPLE_COUNT - image_count
                val_dict[category_path.split("/")[-1]] = required_count
    return train_dict, test_dict, val_dict

In [ ]:
## ➕ Add the Augmented Images to the Balanced Dataset to Balance the Number of Images

## Get the missing number of images
train_dict, test_dict, val_dict = get_missing_images_count(RNN_BALANCED_PATH)

## Print the missing values

def print_dict(dict_name, dataset_type):
    print(f"Missing values in {dataset_type}")
    print("----------------------------")
    for key, value in train_dict.items():
        print(f"{key} : {value}")

### print train_dict key, value pairs
print_dict(train_dict, TRAIN_DIRECTORY)

### print test_dict key, value pairs
print_dict(test_dict, TEST_DIRECTORY)

### print val_dict key, value pairs
print_dict(val_dict, VALIDATION_DIRECTORY)

In [ ]:
## Function to copy the missing number of images randomly from the augmented dataset

def copy_missing_images_from_augmented(src_path, dest_path):
    # Get the missing image count
    train_dict, test_dict, val_dict = get_missing_images_count(CNN_BALANCED_PATH)

    # Dictionary mapping dataset types to their corresponding missing counts
    missing_counts = {
        TRAIN_DIRECTORY: train_dict,
        TEST_DIRECTORY: test_dict,
        VALIDATION_DIRECTORY: val_dict,
    }

    for dataset_type in SUB_DIRECTORIES:
        src_dir_type = os.path.join(src_path, dataset_type)
        dest_dir_type = os.path.join(dest_path, dataset_type)

        # Pass the dictionary for the current dataset type
        copy_images_for_dataset_type(src_dir_type, dest_dir_type, missing_counts[dataset_type])

## Helper function to copy images for a specific dataset type

def copy_images_for_dataset_type(src_dir, dest_dir, missing_dict):
    for key, value in missing_dict.items():
        if value == 0:
            continue
        src_cat_dir = os.path.join(src_dir, key)
        dest_cat_dir = os.path.join(dest_dir, key)

        # Get all available images in the source category directory
        images = [img for img in os.listdir(src_cat_dir) if img.endswith(('jpg', 'jpeg'))]

        # Randomly select the missing number of images
        selected_images = random.sample(images, min(value, len(images)))

        # Copy the selected images to the destination directory
        for img in selected_images:
            shutil.copy(os.path.join(src_cat_dir, img), dest_cat_dir)

In [ ]:
## Copy the missing images from augmented images

copy_missing_images_from_augmented(AUGMENTED_PATH, RNN_BALANCED_PATH)

## Count the number of images again

train_dict, test_dict, val_dict = get_missing_images_count(RNN_BALANCED_PATH)

### print train_dict key, value pairs
print_dict(train_dict, TRAIN_DIRECTORY)

### print test_dict key, value pairs
print_dict(test_dict, TEST_DIRECTORY)

### print val_dict key, value pairs
print_dict(val_dict, VALIDATION_DIRECTORY)

In [ ]:
### Image augmentation that lack the image count

In [ ]:
def load_images_to_df(base_path, dataset_type, sample_size):
    path = os.path.join(base_path, dataset_path)
    image_dir = Path(path)

    file_paths = list(image_dir.glob(r'**/*.jpg'))
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], file_paths))

    file_paths = pd.Series(file_paths, name='Path').astype(str)
    labels = pd.Series(labels, name='Label')

    image_df = pd.concat([file_paths, labels], axis=1)

    samples =[]
    for record in image_df['Label'].unique():
        samples.append(image_df[image_df['Label'] == record].sample(sample_size, random_state=42))
    image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=42).reset_index(drop=True)

    return image_df

In [ ]:
image_df_train = load_images_to_df(CNN_BALANCED_PATH, TRAIN_DIRECTORY, TRAIN_DATASET_SAMPLE_COUNT)

image_df_train.head(10)

In [ ]:
## Show the number of images to show there are no biases in the image set

pie = image_df_train['Label'].value_counts()
pie.plot(kind='pie', autopct='%1.2f%%')
plt.ylabel('')
plt.show()

In [ ]:
## 🖥️ Display Images of the Dataset

### Define figure and axes
fig, axes = plt.subplots(nrows=6, ncols=5, figsize=(15,8), subplot_kw={'xticks':[], 'yticks':[]})

### Display 30 images of the dataset
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df_train.Path[i], -1))
    ax.set_title(image_df_train.Label[i])
plt.tight_layout()
plt.show()

In [ ]:
## Load the Test and Validation Data Frames

image_df_test = load_images_to_df(CNN_BALANCED_PATH, TEST_DIRECTORY, TEST_DATASET_SAMPLE_COUNT)
image_df_test.head(10)

In [ ]:
image_df_val = load_images_to_df(CNN_BALANCED_PATH, VALIDATION_DIRECTORY, VALIDATION_DATASET_SAMPLE_COUNT)
image_df_val.head(10)

In [ ]:
def preprocess_images(df, img_height=IMG_HEIGHT, img_width=IMG_WIDTH):
    images = []
    labels = []
    for index, row in df.iterrows():
        img = tf.keras.utils.load_img(row['Path'], target_size=(img_height, img_width))
        img_array = tf.keras.utils.img_to_array(img)/255.0 # Normalize
        img_array = img_array.reshape(img_height, -1) # Flatten into rows(sequential data)
        images.append(img_array)
        labels.append(row['Label'])
    return np.array(images), np.array(labels)

In [ ]:
# Load training data
train_images, train_labels = preprocess_images(image_df_train)
val_images, val_labels = preprocess_images(image_df_val)
test_images, test_labels = preprocess_images(image_df_test)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(test_labels)
test_labels = label_encoder.transform(test_labels)

In [ ]:
# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, NUM_CLASSES)
val_labels = to_categorical(val_labels, NUM_CLASSES)
test_labels = to_categorical(test_labels, NUM_CLASSES)

In [ ]:
def build_model(hp):
    model = Sequential()

    # First LSTM layer with tunable number of units
    model.add(LSTM(units=hp.Int('units_lstm_1', min_value=64, max_value=256, step=64),
    input_shape=(IMG_HEIGHT, IMG_WIDTH *3),
    return_sequences=True))

    # Second LSTM layer with tunable number of units
    model.add(LSTM(units=hp.Int('units_lstm_2', min_value=32, max_value=128, step=32)))

    # Dropout layer with tunable rate
    moel.add(Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense layer with tunable number of units
    model.add(Dense(units=hp.Int('units_dense', min_value=16, max_value=64, step=16), activation='relu'))

    # Output layer
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    model.compile(
        optimizer=tf.keras.optimizers.legacy.Adam(
            learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
# Initialize the tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='rnn_tuning',
    project_name='image_classification'
)

In [ ]:
# Define an early stopping callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# Run the hyperparameter search
tuner.search(
    train_images, train_labels,
    validation_data=(val_images, val_labels),
    epochs=20,
    callbacks=[stop_early],
    batch_size=BATCH_SIZE
)

In [ ]:
# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Best hyperparameters:
- Units in LSTM Layer 1: {best_hps.get('units_lstm_1')}
- Units in LSTM Layer 2: {best_hps.get('units_lstm_2')}
- Dropout Rate: {best_hps.get('dropout_rate')}
- Units in Dense Layer: {best_hps.get('units_dense')}
- Learning Rate: {best_hps.get('learning_rate')}
""")

In [ ]:
# Build the best model and retrain
best_model = tuner.hypermodel.build(best_hps)

In [ ]:
history = best_model.fit(train_images, train_labels, validation_data=(val_images, val_labels),
                    batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[stop_early])

In [ ]:
# Evaluate the model
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy:.2f}")

In [ ]:
# Classification Report
predictions = best_model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels, axis=1)
print(classification_report(true_classes, predicted_classes, target_names=label_encoder.classes_))

In [ ]:
# Accuracy Plot
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# Loss Plot
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()